In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- Es importante tener en cuenta que los precios pasados no garantizan resultados futuros, por eso podemos utilizar:

Técnicas de aprendizaje automático para predecir los precios de las acciones en el futuro y se explorará si hay algún patrón en el comportamiento de las acciones según el día de la semana.

In [2]:
#Librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression
import datetime as dt

In [3]:
# Obtener datos de Yahoo Finance
tickers_sec= ["XLC", "XLY", "XLP", "XLE", "XLF", "XLV", "XLI", "XLK", "XLU", "XLB"]
data_pred = yf.download(tickers_sec, start="2000-01-01", end="2023-06-01")

[*********************100%***********************]  10 of 10 completed


In [4]:
#Filtro por precio ajustado
data=data_pred["Adj Close"]

In [5]:
# Vistazo
data.head()

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
Date,,,,,,,,,,
2000-01-03,15.332277,NaN,14.677699,11.641031,18.901960,42.116337,13.171596,12.086847,21.487967,22.854641
2000-01-04,15.184673,NaN,14.401417,11.132135,18.379755,39.979652,12.800688,11.722251,21.000357,22.167093
2000-01-05,15.811992,NaN,14.781306,11.044669,18.297829,39.386131,13.026845,12.018059,20.811966,21.894453
2000-01-06,16.217905,NaN,15.351150,11.529708,18.543575,38.080391,13.271099,11.997420,20.878458,22.143391
2000-01-07,16.208668,NaN,15.515188,11.720544,19.250101,38.745125,14.112418,12.100607,21.133339,23.186548


In [8]:
# Reemplazar los valores NaN por 0
data = data.fillna(0)

In [9]:
# Crear un objeto de regresión lineal
reg = LinearRegression()

In [10]:
# Crear un diccionario vacío para almacenar los DataFrames de predicciones para cada ticker
predictions_dict = {}

# Iterar sobre cada ticker del sector
for ticker in tickers_sec:
    # Seleccionar el ticker actual
    ticker_data = data[ticker]
    
    # Definir la variable de entrada X (fechas)
    X = np.array(ticker_data.index.astype(int) // 10**9).reshape(-1,1)
    
    # Definir la variable de salida Y (precios ajustados)
    Y = np.array(ticker_data.values)
    
    # Entrenar un modelo de regresión lineal
    reg = LinearRegression().fit(X, Y)
    
    # Definir las fechas futuras para las que se quieren hacer predicciones
    future_dates = pd.date_range(start="2023-06-01", periods=365, freq="D")
    future_date_values = np.array(future_dates.astype(int) // 10**9)
    
    # Hacer predicciones de precios para las fechas futuras utilizando el modelo de regresión lineal
    price_predictions = reg.predict(future_date_values.reshape(-1,1))
    
    # Crear un DataFrame que contenga las fechas futuras y las predicciones de precios
    predictions_df = pd.DataFrame({
        "Date": future_dates,
        "Price Prediction": price_predictions.flatten(),
        "Ticker": ticker
    })
    
    # Agregar el DataFrame de predicciones para el ticker actual al diccionario
    predictions_dict[ticker] = predictions_df
    
# Combinar todos los DataFrames en uno solo
predictions_combined = pd.concat(predictions_dict.values(), ignore_index=True)

# Imprimir el DataFrame combinado
print(predictions_combined)

           Date  Price Prediction Ticker
0    2023-06-01         41.412634    XLC
1    2023-06-02         41.419475    XLC
2    2023-06-03         41.426316    XLC
3    2023-06-04         41.433157    XLC
4    2023-06-05         41.439998    XLC
...         ...               ...    ...
3645 2024-05-26         70.017704    XLB
3646 2024-05-27         70.025220    XLB
3647 2024-05-28         70.032735    XLB
3648 2024-05-29         70.040251    XLB
3649 2024-05-30         70.047767    XLB

[3650 rows x 3 columns]


In [11]:
# Nombres unicos de la columna Ticker
predictions_combined["Ticker"].unique()

array(['XLC', 'XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLK', 'XLU',
       'XLB'], dtype=object)

In [12]:
# Vistazo
predictions_combined

,Date,Price Prediction,Ticker
0,2023-06-01,41.412634,XLC
1,2023-06-02,41.419475,XLC
2,2023-06-03,41.426316,XLC
3,2023-06-04,41.433157,XLC
4,2023-06-05,41.439998,XLC
...,...,...,...
3645,2024-05-26,70.017704,XLB
3646,2024-05-27,70.025220,XLB
3647,2024-05-28,70.032735,XLB
3648,2024-05-29,70.040251,XLB


In [14]:
# Guardo datos de  predicción juntas
#predictions_combined.to_excel("/content/drive/MyDrive/EDA/Prediccion.xlsx", index=False)

En el DF predictions_combined se encuentran las predicciones juntas, asi que las separamos:

In [15]:
xly_df = predictions_combined.loc[predictions_combined["Ticker"] == "XLY"]
xlk_df = predictions_combined.loc[predictions_combined["Ticker"] == "XLK"]
xlv_df = predictions_combined.loc[predictions_combined["Ticker"] == "XLV"]
xli_df = predictions_combined.loc[predictions_combined["Ticker"] == "XLI"]
xlc_df = predictions_combined.loc[predictions_combined["Ticker"] == "XLC"]
xlp_df = predictions_combined.loc[predictions_combined["Ticker"] == "XLP"]
xle_df = predictions_combined.loc[predictions_combined["Ticker"] == "XLE"]
xlf_df = predictions_combined.loc[predictions_combined["Ticker"] == "XLF"]
xlu_df = predictions_combined.loc[predictions_combined["Ticker"] == "XLU"]
xlb_df = predictions_combined.loc[predictions_combined["Ticker"] == "XLB"]

In [29]:
# Guardo datos de  predicción por separado
#xly_df.to_excel("/content/drive/MyDrive/EDA/SECTORES/XLY.xlsx", index=False)
#xlk_df.to_excel("/content/drive/MyDrive/EDA/SECTORES/XLK.xlsx", index=False)
#xlv_df.to_excel("/content/drive/MyDrive/EDA/SECTORES/XLV.xlsx", index=False)
#xli_df.to_excel("/content/drive/MyDrive/EDA/SECTORES/XLI.xlsx", index=False)
#xlc_df.to_excel("/content/drive/MyDrive/EDA/SECTORES/XLC.xlsx", index=False)
#xlp_df.to_excel("/content/drive/MyDrive/EDA/SECTORES/XLP.xlsx", index=False)
#xle_df.to_excel("/content/drive/MyDrive/EDA/SECTORES/XLE.xlsx", index=False)
#xlf_df.to_excel("/content/drive/MyDrive/EDA/SECTORES/XLF.xlsx", index=False)
#xlu_df.to_excel("/content/drive/MyDrive/EDA/SECTORES/XLU.xlsx", index=False)
#xlb_df.to_excel("/content/drive/MyDrive/EDA/SECTORES/XLB.xlsx", index=False)

Vistazo de cada una:

In [17]:
xly_df

,Date,Price Prediction,Ticker
365,2023-06-01,133.458730,XLY
366,2023-06-02,133.475845,XLY
367,2023-06-03,133.492960,XLY
368,2023-06-04,133.510075,XLY
369,2023-06-05,133.527191,XLY
...,...,...,...
725,2024-05-26,139.620199,XLY
726,2024-05-27,139.637314,XLY
727,2024-05-28,139.654429,XLY
728,2024-05-29,139.671545,XLY


In [18]:
xlk_df

,Date,Price Prediction,Ticker
2555,2023-06-01,98.590892,XLK
2556,2023-06-02,98.603662,XLK
2557,2023-06-03,98.616432,XLK
2558,2023-06-04,98.629202,XLK
2559,2023-06-05,98.641972,XLK
...,...,...,...
2915,2024-05-26,103.188078,XLK
2916,2024-05-27,103.200848,XLK
2917,2024-05-28,103.213618,XLK
2918,2024-05-29,103.226388,XLK


In [19]:
xlv_df

,Date,Price Prediction,Ticker
1825,2023-06-01,104.897807,XLV
1826,2023-06-02,104.910695,XLV
1827,2023-06-03,104.923583,XLV
1828,2023-06-04,104.936471,XLV
1829,2023-06-05,104.949359,XLV
...,...,...,...
2185,2024-05-26,109.537475,XLV
2186,2024-05-27,109.550363,XLV
2187,2024-05-28,109.563251,XLV
2188,2024-05-29,109.576139,XLV


In [20]:
xli_df

,Date,Price Prediction,Ticker
2190,2023-06-01,82.391386,XLI
2191,2023-06-02,82.400980,XLI
2192,2023-06-03,82.410574,XLI
2193,2023-06-04,82.420168,XLI
2194,2023-06-05,82.429762,XLI
...,...,...,...
2550,2024-05-26,85.845253,XLI
2551,2024-05-27,85.854848,XLI
2552,2024-05-28,85.864442,XLI
2553,2024-05-29,85.874036,XLI


In [21]:
xlc_df

,Date,Price Prediction,Ticker
0,2023-06-01,41.412634,XLC
1,2023-06-02,41.419475,XLC
2,2023-06-03,41.426316,XLC
3,2023-06-04,41.433157,XLC
4,2023-06-05,41.439998,XLC
...,...,...,...
360,2024-05-26,43.875360,XLC
361,2024-05-27,43.882201,XLC
362,2024-05-28,43.889042,XLC
363,2024-05-29,43.895883,XLC


In [22]:
xlp_df

,Date,Price Prediction,Ticker
730,2023-06-01,62.790045,XLP
731,2023-06-02,62.797302,XLP
732,2023-06-03,62.804558,XLP
733,2023-06-04,62.811815,XLP
734,2023-06-05,62.819072,XLP
...,...,...,...
1090,2024-05-26,65.402520,XLP
1091,2024-05-27,65.409777,XLP
1092,2024-05-28,65.417033,XLP
1093,2024-05-29,65.424290,XLP


In [24]:
xle_df

,Date,Price Prediction,Ticker
1095,2023-06-01,65.721151,XLE
1096,2023-06-02,65.726917,XLE
1097,2023-06-03,65.732682,XLE
1098,2023-06-04,65.738448,XLE
1099,2023-06-05,65.744213,XLE
...,...,...,...
1455,2024-05-26,67.796738,XLE
1456,2024-05-27,67.802504,XLE
1457,2024-05-28,67.808269,XLE
1458,2024-05-29,67.814035,XLE


In [25]:
xlf_df

,Date,Price Prediction,Ticker
1460,2023-06-01,26.687678,XLF
1461,2023-06-02,26.689750,XLF
1462,2023-06-03,26.691822,XLF
1463,2023-06-04,26.693895,XLF
1464,2023-06-05,26.695967,XLF
...,...,...,...
1820,2024-05-26,27.433714,XLF
1821,2024-05-27,27.435786,XLF
1822,2024-05-28,27.437859,XLF
1823,2024-05-29,27.439931,XLF


In [26]:
xlu_df

,Date,Price Prediction,Ticker
2920,2023-06-01,59.071503,XLU
2921,2023-06-02,59.078207,XLU
2922,2023-06-03,59.084911,XLU
2923,2023-06-04,59.091616,XLU
2924,2023-06-05,59.098320,XLU
...,...,...,...
3280,2024-05-26,61.485033,XLU
3281,2024-05-27,61.491737,XLU
3282,2024-05-28,61.498442,XLU
3283,2024-05-29,61.505146,XLU


In [27]:
xlb_df

,Date,Price Prediction,Ticker
3285,2023-06-01,67.312065,XLB
3286,2023-06-02,67.319581,XLB
3287,2023-06-03,67.327096,XLB
3288,2023-06-04,67.334612,XLB
3289,2023-06-05,67.342128,XLB
...,...,...,...
3645,2024-05-26,70.017704,XLB
3646,2024-05-27,70.025220,XLB
3647,2024-05-28,70.032735,XLB
3648,2024-05-29,70.040251,XLB
